# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Ground-predictions" data-toc-modified-id="Ground-predictions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ground predictions</a></div><div class="lev2 toc-item"><a href="#PVLib-Clearsky" data-toc-modified-id="PVLib-Clearsky-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>PVLib Clearsky</a></div>

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn import tree

import pytz
import itertools
import visualize
import utils
import pydotplus

from sklearn import metrics

import pvlib
import cs_detection
# import visualize
# from bokeh.plotting import output_notebook
# output_notebook()

import visualize_plotly as visualize

from IPython.display import Image

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4)
%matplotlib notebook

# Ground predictions

## PVLib Clearsky

Only making ground predictions using PVLib clearsky model and statistical model.  NSRDB model won't be available to ground measurements.

In [2]:
nsrdb = cs_detection.ClearskyDetection.read_pickle('abq_nsrdb_1.pkl.gz')
nsrdb.df.index = nsrdb.df.index.tz_convert('MST')

In [3]:
nsrdb.time_from_solar_noon('Clearsky GHI pvlib', 'tfn')

In [4]:
ground = cs_detection.ClearskyDetection.read_pickle('abq_ground_1.pkl.gz')
ground.df.index = ground.df.index.tz_convert('MST')

We will reduce the frequency of ground based measurements to match NSRDB.

In [5]:
ground.intersection(nsrdb.df.index)

In [6]:
nsrdb.scale_model('GHI', 'Clearsky GHI pvlib', 'sky_status')

In [7]:
nsrdb.calc_all_window_metrics(3, 30, col1='GHI', col2='Clearsky GHI pvlib', overwrite=True)

In [8]:
ground.calc_all_window_metrics(3, 30, col1='GHI', col2='Clearsky GHI pvlib', overwrite=True)

In [9]:
feature_cols = [
'GHI',
'Clearsky GHI pvlib',
'tfn',
'GHI mean',
'GHI std',
'GHI max',
'GHI min',
'GHI range',
'Clearsky GHI pvlib mean',
'Clearsky GHI pvlib std',
'Clearsky GHI pvlib max',
'Clearsky GHI pvlib min',
'Clearsky GHI pvlib range',
'GHI gradient',
'GHI gradient mean',
'GHI gradient std',
'GHI gradient max',
'GHI gradient min',
'GHI gradient range',
'GHI gradient second',
'GHI gradient second mean',
'GHI gradient second std',
'GHI gradient second max',
'GHI gradient second min',
'GHI gradient second range',
'Clearsky GHI pvlib gradient',
'Clearsky GHI pvlib gradient mean',
'Clearsky GHI pvlib gradient std',
'Clearsky GHI pvlib gradient max',
'Clearsky GHI pvlib gradient min',
'Clearsky GHI pvlib gradient second',
'Clearsky GHI pvlib gradient second mean',
'Clearsky GHI pvlib gradient second std',
'Clearsky GHI pvlib gradient second max',
'Clearsky GHI pvlib gradient second min',
'abs_ideal_ratio_diff',
'abs_ideal_ratio_diff mean',
'abs_ideal_ratio_diff std',
'abs_ideal_ratio_diff max',
'abs_ideal_ratio_diff min',
'abs_ideal_ratio_diff range',
'abs_ideal_ratio_diff gradient',
'abs_ideal_ratio_diff gradient mean',
'abs_ideal_ratio_diff gradient std',
'abs_ideal_ratio_diff gradient max',
'abs_ideal_ratio_diff gradient min',
'abs_ideal_ratio_diff gradient range',
'abs_ideal_ratio_diff gradient second',
'abs_ideal_ratio_diff gradient second mean',
'abs_ideal_ratio_diff gradient second std',
'abs_ideal_ratio_diff gradient second max',
'abs_ideal_ratio_diff gradient second min',
'abs_ideal_ratio_diff gradient second range',
'abs_diff',
'abs_diff mean',
'abs_diff std',
'abs_diff max',
'abs_diff min',
'abs_diff range',
'abs_diff gradient',
'abs_diff gradient mean',
'abs_diff gradient std',
'abs_diff gradient max',
'abs_diff gradient min',
'abs_diff gradient range',
'abs_diff gradient second',
'abs_diff gradient second mean',
'abs_diff gradient second std',
'abs_diff gradient second max',
'abs_diff gradient second min',
'abs_diff gradient second range',
'GHI line length',
'Clearsky GHI pvlib line length',
'GHI Clearsky GHI pvlib line length ratio',
# 'GHI bpct change',
# 'GHI bpct change mean', 'GHI bpct change std', 'GHI bpct change max', 'GHI bpct change min', 'GHI bpct change range',
# 'Clearsky GHI pvlib bpct change',
# 'Clearsky GHI pvlib bpct change mean', 'Clearsky GHI pvlib bpct change std', 'Clearsky GHI pvlib bpct change max', 'Clearsky GHI pvlib bpct change min', 'Clearsky GHI pvlib bpct change range',
# 'GHI Clearsky GHI pvlib bpct change ratio', 
# 'GHI Clearsky GHI pvlib bpct change ratio mean', 'GHI Clearsky GHI pvlib bpct change ratio std', 'GHI Clearsky GHI pvlib bpct change ratio max', 'GHI Clearsky GHI pvlib bpct change ratio min', 'GHI Clearsky GHI pvlib bpct change ratio range' 
]

target_cols = ['sky_status']

In [10]:
ground.trim_dates('10-01-2015', '10-08-2015')

In [11]:
ground.time_from_solar_noon('Clearsky GHI pvlib', 'tfn')

In [12]:
train = cs_detection.ClearskyDetection(nsrdb.df)
test = cs_detection.ClearskyDetection(ground.df)

In [13]:
from sklearn import ensemble
# clf = ensemble.RandomForestClassifier(class_weight='balanced', n_estimators=100, max_leaf_nodes=40)  # max_leaf_nodes=30, n_estimators=100)
clf = tree.DecisionTreeClassifier(class_weight='balanced', min_samples_leaf=.0025)
clf = ensemble.RandomForestClassifier(class_weight='balanced', min_samples_leaf=.003, n_estimators=24, n_jobs=-1)

In [14]:
# nsrdb.df = nsrdb.df[nsrdb.df['GHI'] > 0]

In [15]:
# train.df = train.df[train.df['Clearsky GHI pvlib'] > 0]

In [16]:
clf.fit(train.df[feature_cols].values, train.df[target_cols].values.flatten())

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=0.003, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=24, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
# pred = clf.predict(test.df[feature_cols].values)
pred = test.iter_predict_daily(feature_cols, 'GHI', 'Clearsky GHI pvlib', clf, 3)
pred = pred.astype(bool)

1 1
2 0.760943948086
3 1.25235552549
4 0.798495296168
5 1.25235553021
6 0.798495296168
7 1.25235553021
8 0.798495296168
9 1.25235553021
10 0.798495296168
11 1.25235553021
12 0.798495296168
13 1.25235553021
14 0.798495296168
15 1.25235553021
16 0.798495296168
17 1.25235553021
18 0.798495296168
19 1.25235553021
20

/Users/benellis/duramat/clearsky_detection/cs_detection.py:919: RuntimeWarning:

Scaling did not converge.



 0.798495296168
1 1
2 0.615937393287
3 1.08494859102
4 0.994672468006
5 1.0
1 1
2 0.450064215681
3 1.49628239895
4 0.999076550573
5 1.0
1 1
2 0.425308971682
3 0.996934289769
4 1.0
1 1
2 0.46373238808
3 0.984590799756
4 1.0
1 1
2 0.731005461362
3 1.16756316203
4 0.998146424266
5 1.0
1 1
2 0.480003176014
3 0.99990297702
4 1.0


In [18]:
train.intersection(test.df.index)

In [19]:
vis = visualize.Visualizer()

In [20]:
vis.add_line_ser(test.df['GHI'], 'GHI')
vis.add_line_ser(test.df['Clearsky GHI pvlib'], 'GHI_cs')
vis.add_circle_ser(test.df[(train.df['sky_status'] == 0) & (pred)]['GHI'], 'ML clear only')
vis.add_circle_ser(test.df[(train.df['sky_status'] == 1) & (~pred)]['GHI'], 'NSRDB clear only')
vis.add_circle_ser(test.df[(train.df['sky_status'] == 1) & (pred)]['GHI'], 'ML+NSRDB clear only')

In [21]:
vis.show()

In [22]:
ground = cs_detection.ClearskyDetection.read_pickle('abq_ground_1.pkl.gz')

In [23]:
ground.df.index = ground.df.index.tz_convert('MST')

In [24]:
ground.trim_dates('11-01-2015', '11-08-2015')

In [25]:
ground.time_from_solar_noon('Clearsky GHI pvlib', 'tfn')

In [26]:
test = ground

In [27]:
# pred = clf.predict(test.df[feature_cols].values)
pred = test.iter_predict_daily(feature_cols, 'GHI', 'Clearsky GHI pvlib', clf, 10, smooth=True)
pred = pred.astype(bool)

1 1
2 0.96823772152
3 1.00001799625
4 1.00000363261
5 1.0
1 1
2 0.981096141343
3 1.00037238874
4 1.00003238736
5 1.00000201791
6 1.0
1 1
2 0.966140280083
3 0.999950478817
4 0.999993546723
5 1.0
1 1
2 1.01458910522
3 1.0147643844
4 1.01102571483
5 1.01083712433
6 1.0169021844
7 1.01117128145
8 1.00792395592
9 1.00064456288
10 1.0
1 1
2 1.01093492869
3 1.00435869478
4 1.00180983218
5 1.00004797258
6 1.00011829333
7 1.0001752301
8 1.0000127986
9 1.0
1 1
2 0.975866248116
3 1.00018760319
4 1.0
1 1
2 0.977930329067
3 1.00005262762
4 1.0


In [28]:
vis = visualize.Visualizer()

In [29]:
vis.add_line_ser(test.df['GHI'], 'GHI')
vis.add_line_ser(test.df['Clearsky GHI pvlib'], 'GHI_cs')
vis.add_circle_ser(test.df[(test.df['sky_status pvlib'] == 0) & (pred)]['GHI'], 'ML clear only')
vis.add_circle_ser(test.df[(test.df['sky_status pvlib'] == 1) & (~pred)]['GHI'], 'PVLib clear only')
vis.add_circle_ser(test.df[(test.df['sky_status pvlib'] == 1) & (pred)]['GHI'], 'ML+PVLib clear only')

In [30]:
vis.show()